# Semi-supervised learning on renders 


In [23]:
# !pip install "tensorflow_hub>=0.6.0"
# !pip install "tensorflow>=2.0.0" uncomment this if tensorflow not found

from tensorflow.keras.layers import Input, Dense, Flatten, Activation, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K


In [24]:
# Process Image
    
IMG_WIDTH, IMG_HEIGHT = 224,224
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  img_resize = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
  return img_resize

def process_path(file_path):
  # label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, img
  # return img, label

In [25]:
list_ds = tf.data.Dataset.list_files(str('../render/render*'))
for f in list_ds.take(5):
  print(f)    

tf.Tensor(b'../render/render2288.png', shape=(), dtype=string)
tf.Tensor(b'../render/render344.png', shape=(), dtype=string)
tf.Tensor(b'../render/render1306.png', shape=(), dtype=string)
tf.Tensor(b'../render/render9642.png', shape=(), dtype=string)
tf.Tensor(b'../render/render5508.png', shape=(), dtype=string)


In [26]:
import tensorflow.keras.backend as K
K.tensorflow_backend._get_available_gpus()

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score
    

ImportError: No module named 'keras'

In [53]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 256
image_count = 10394
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
for item, mirror_item in labeled_ds.take(1):
    ImgShape = item.shape
# df = pd.DataFrame(labeled_ds)
# df.info()

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()
  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()
    

num_class = 10    
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
def classifier_func(x):
    return x+x*K.one_hot(K.argmax(x, axis=1), num_classes=num_class)

In [54]:
def GetModel(input_shape):
    input_img = Input(shape=(224, 224, 3))

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    x = Conv2D(2, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    
    # Output Shape: 4x4x8
    x = Conv2D(2, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    # Output Shape: 28x28x1
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer="adadelta", loss="mse", metrics=['acc']) # may add f1_score later
    autoencoder.summary()
    return autoencoder

def GetModel2(input_shape):
    input_img = Input(shape=(224, 224, 3))
    #Encoder:
    conv_1 = Conv2D(16, (3,3), strides=(1,1))(input_img)
    act_1 = Activation('relu')(conv_1)
    maxpool_1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(act_1)

    conv_2 = Conv2D(4, (3,3), strides=(1,1), padding='same')(maxpool_1)
    act_2 = Activation('relu')(conv_2)
    maxpool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(act_2)
    # Output Shape: 6x6x64
    conv_3 = Conv2D(1, (3,3), strides=(1,1), padding='same')(maxpool_2)
    act_3 = Activation('relu')(conv_3)
    maxpool_3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(act_3)

    flat_1 = Flatten()(maxpool_3)

    fc_1 = Dense(256)(flat_1)
    act_3 = Activation('relu')(fc_1)

    fc_2 = Dense(128)(act_3)
    act_4 = Activation('relu')(fc_2)

    fc_3 = Dense(num_class)(act_4)

    act_class = Lambda(classifier_func, output_shape=(num_class,))(fc_3)
    # Output Shape: 10

    #Decoder:
    fc_4 = Dense(256)(act_class)
    act_5 = Activation('relu')(fc_4)

    fc_5 = Dense(2304)(act_5)
    act_6 = Activation('relu')(fc_5)
    reshape_1 = Reshape((6,6,16))(act_6)
    
    
    upsample_1 = UpSampling2D((2, 2))(reshape_1)
    deconv_1 = Conv2DTranspose(16, (3, 3), strides=(1, 1))(upsample_1)
    act_7 = Activation('relu')(deconv_1)

    upsample_2 = UpSampling2D((2, 2))(act_7)
    deconv_2 = Conv2DTranspose(8, (3, 3), strides=(1, 1))(upsample_2)
    act_8 = Activation('relu')(deconv_2)
    
    upsample_3 = UpSampling2D((2, 2))(act_8)
    deconv_3 = Conv2DTranspose(1, (3, 3), strides=(1, 1))(upsample_3)
    act_9 = Activation('relu')(deconv_3)

    conv_3 = Conv2D(3, (3, 3), strides=(1, 1))(act_9)
    act_10 = Activation('sigmoid')(conv_3)
    # Output Shape: 28x28x1

    autoencoder = Model(input_img, act_10)
    autoencoder.summary()
    return autoencoder

Dataset_train = prepare_for_training(labeled_ds)

In [55]:
autoencoder = GetModel(input_shape=ImgShape)
Dataset_train

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 112, 112, 8)       1160      
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 56, 56, 8)         0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 56, 56, 4)         292       
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 28, 28, 4)         0  

<PrefetchDataset shapes: ((None, 224, 224, 3), (None, 224, 224, 3)), types: (tf.float32, tf.float32)>

In [57]:

# Use unlabeled data to train the Encoder
# autoencoder.fit(X_train, X_train, steps_per_epoch=16, epochs=10, verbose=1,
#                shuffle=True)

# Data_train = tf.data.Dataset.from_tensor_slices((X_train, X_train))

batch_stats_callback = CollectBatchStats()
history = autoencoder.fit_generator(Dataset_train, epochs=3,
                              steps_per_epoch=STEPS_PER_EPOCH,
                              callbacks = [batch_stats_callback])

Train for 41.0 steps
Epoch 1/3
41/41 [==============================] - 287s 7s/step - loss: 0.1867 - acc: 0.7931
Epoch 2/3
41/41 [==============================] - 263s 6s/step - loss: 0.1865 - acc: 0.7969
Epoch 3/3
41/41 [==============================] - 262s 6s/step - loss: 0.1866 - acc: 0.7872


OSError: Unable to create file (unable to open file: name = './saved_model/autoencoder.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [58]:
autoencoder.save('./saved_model/autoencoder.h5')

In [ ]:
autoencoder = tf.keras.models.load_model('./saved_model/autoencoder.h5')

In [59]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])
hidden_representation.add(autoencoder.layers[3])
hidden_representation.add(autoencoder.layers[4])
hidden_representation.add(autoencoder.layers[5])
hidden_representation.add(autoencoder.layers[6])
hidden_representation.add(autoencoder.layers[7])
hidden_representation.add(autoencoder.layers[8])


data_labeled = pd.read_csv('./data/dataset_labeled.csv')
X_labeled = X_train[0:799]

X_rep = hidden_representation.predict(X_labeled)

# X_labeled = vectorizer.fit_transform(X_labeled)
y_rep = data_labeled['NewCategory']

NameError: name 'X_train' is not defined